# 1) Data Import and Important libraries

In [1]:
#Importing important libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Getting the data from google drive
!gdown --id '1ZLsD1w5YXX0QgfupwC0gYTOGASa-nbkR'

In [7]:
data=pd.read_csv('/content/zameen-updated.csv')
data.head()

# 2) Statical Analysis

In [8]:
data.describe()

# 2.1 Insites
This data tell us that in some there are some outliers are there.
#### Like
The price parameter has mean of 1.776576e+07 but the max value is 2.000000e+09 and 75% values are 1.950000e+07. That's means it has outliers that are making this field unstructured.


# 3) Data Cleaning

In [9]:
data.head(2)

### 3.1 First let's understand the dataset
We don't need 

1. proporty_id
2. location_id
3. page_url
4. area
5. area Category 

because these all have nothing to do with our model. That is why drop these parameters.

In [10]:
#columns drop
data=data.drop(['property_id','location_id','page_url','area','Area Category'],axis=1)
data.head()

### 3.2 Null values

In [12]:
data.isnull().sum()

Here we have 44071 null values from agency and agent only. That is the 26 percentage of total dataset.

In [26]:
total_values=data.index.stop
null_values=data['agency'].isnull().sum()

percentage=null_values/total_values
percentage

In [34]:
#Unique values of agency and agent
agency_uniques=data['agency'].unique()[0:-1]
list(agency_uniques)

In [36]:
#For agent
list(data['agent'].unique())

We can fill these null values with 'Self' word because they all do their work by their own.

In [39]:
data['agency'].fillna('Self',inplace=True)
data['agent'].fillna('Self',inplace=True)

In [40]:
data.isnull().sum()

Check point

In [86]:
data1=data.copy()

### 3.3 Duplicate Values

In [87]:
data1.duplicated().sum()

Delete these duplicate values they might effect our model.

In [88]:
data1.drop_duplicates(inplace=True)
data1.duplicated().sum()

### 3.4 Data Types

In [89]:
data1.info()

In [90]:
data1.head(1)

In [91]:
#Now let's change the date into date format by pandas.to_datatime()
data1['date_added']=pd.to_datetime(data1['date_added'])

In [92]:
data1.info()

In [93]:
data1.head()

Change the Area Size from Marla and Kanal into only Marla

In [99]:
Area_in_marla=data1[['Area Type','Area Size']].apply(lambda x: x['Area Size']*20 if x['Area Type'] in ['Kanal'] else x, axis=1)
Area_in_marla.drop(['Area Type'],axis=1,inplace=True)
Area_in_marla.rename(columns={'Area Size':'Area_in_Marla'},inplace=True)
data1=pd.concat([data1,Area_in_marla],axis=1)
data1

check point

In [100]:
data2=data1.copy()

3.5 Check for Correlation between the data

In [104]:
sns.heatmap(data2.corr(),annot=True)

Price is our Dependent Variable so the correlation with price of baths,bedrooms,area_in_marla has really good correlation so we will drop others column after doing EDA.

In [106]:
data2.drop(['Area Type','Area Size'],axis=1,inplace=True)

In [108]:
data2.head()

Save that dataset for now to do EDA

In [111]:
data2.to_csv('EDA_dataset.csv')

### 3.6 Drop
Let's drop the unwanted features

In [114]:
data2.drop(['province_name',
            'agent',
            'agency',
            'date_added',
            'longitude',
            'latitude'],
           axis=1,
           inplace=True)
data2.head()

Check for unique values in each

In [115]:
data2['property_type'].unique()

In [123]:
list(data2['location'].unique())

In [124]:
data2['city'].unique()

In [125]:
data2['purpose'].unique(
)

check point

In [129]:
data3=data2.copy()

Save dataset for model building

Making Dependent and independent features

### 3.7 Outliers Detection

In [156]:
plt.rcParams["figure.figsize"] = [7.50, 3.50]
plt.rcParams["figure.autolayout"] = True

ax = data3[['price', 'baths', 'bedrooms', 'Area_in_Marla']].plot(kind='box', title='boxplot')
plt.show()

3.7.1 Price Outliers removel

In [158]:
sns.distplot(x=data3['price'],kde=False)

In [184]:
#Removing Outliers by using Percentile
min_thresold=data3['price'].quantile(0.01)
max_thresold=data3['price'].quantile(0.90)
min_thresold,max_thresold

In [185]:
data4=data3[(data3['price']>min_thresold)&(data3['price']<max_thresold)]

In [186]:
sns.distplot(x=data4['price'],kde=False)

3.7.2 Baths outliers removel

In [187]:
sns.distplot(x=data4['baths'],kde=False)

In [188]:
min_thresold=data4['baths'].quantile(0.01)
max_thresold=data4['baths'].quantile(0.99)
min_thresold,max_thresold

In [189]:
data5=data4[(data4['baths']>min_thresold)&(data4['baths']<max_thresold)]

In [190]:
sns.distplot(x=data5['baths'],kde=False)

3.7.3 Bedrooms remvoel

In [191]:
sns.distplot(data5['bedrooms'],kde=False)

In [192]:
max_thresold=data5['baths'].quantile(0.99)
max_thresold

In [194]:
data6=data5[(data5['bedrooms']<max_thresold)]

3.7.4 Size in Marla outliers removel

In [195]:
sns.distplot(x=data6['Area_in_Marla'],kde=False)

In [218]:
max_thresold=data6['Area_in_Marla'].quantile(0.999)
max_thresold

In [219]:
data6[data6['Area_in_Marla']>max_thresold]

In [220]:
data7=data6[(data6['Area_in_Marla']<max_thresold)]


In [221]:
sns.distplot(x=data7['Area_in_Marla'],kde=False)

In [223]:
data7.to_csv('cleaned_data_for_model.csv')